In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
from keras.layers import Flatten, Conv2D, MaxPooling2D, Dropout, MaxPool2D, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense
from keras.models import Model, Sequential, load_model
from keras import applications
from keras.utils.np_utils import to_categorical
from keras.optimizers import RMSprop, Adam, SGD
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras import backend as K

In [ ]:
train = pd.read_csv('../input/train.csv')
train.head()

In [ ]:
train['has_cactus'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
import tqdm

img = plt.imread('../input/train/train/'+ train['id'][0])
img.shape

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score

x_train, x_test, y_train, y_test = train_test_split(train['id'], train['has_cactus'], test_size = 0.1, random_state = 32)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
x_train_arr = []
for images in tqdm.tqdm(x_train):
    img = plt.imread('../input/train/train/' + images)
    x_train_arr.append(img)

x_train_arr = np.array(x_train_arr)
print(x_train_arr.shape)

In [ ]:
x_test_arr = []
for images in tqdm.tqdm(x_test):
    img = plt.imread('../input/train/train/' + images)
    x_test_arr.append(img)

x_test_arr = np.array(x_test_arr)
print(x_test_arr.shape)

In [ ]:
x_train_arr = x_train_arr.astype('float32')
x_test_arr = x_test_arr.astype('float32')
x_train_arr = x_train_arr/255
x_test_arr = x_test_arr/255

In [ ]:
print(x_train_arr.shape)
print(x_test_arr.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
from keras.applications.densenet import DenseNet201

conv_base = DenseNet201(
    weights='imagenet',
    include_top=False,
    input_shape=(32,32,3)
)
       
model = Sequential()
model.add(conv_base)
# model.add(Conv2D(filters = 512, kernel_size = (3,3),padding = 'Same', activation ='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(256, use_bias=True))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.5))
# model.add(Dense(128, use_bias=True))
# model.add(BatchNormalization())
# model.add(Activation("relu"))
# model.add(Dropout(0.3))
model.add(Dense(1, activation = "sigmoid"))

In [ ]:
conv_base.Trainable=True

set_trainable=False
for layer in conv_base.layers:
    layer.trainable = True

In [ ]:
model.summary()

In [ ]:
model.compile('rmsprop', loss = "binary_crossentropy", metrics=["accuracy"])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

batch_size = 128
epochs = 30

filepath="weights_resnet.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
learning_rate_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, verbose=1, mode='max', min_delta=0.0, cooldown=0, min_lr=0)
early_stop = EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=25, verbose=1, mode='max', baseline=None, restore_best_weights=True)

train_datagen = ImageDataGenerator(
    rotation_range=20,
    vertical_flip=True,
    horizontal_flip=True)

train_datagen.fit(x_train_arr)


history = model.fit_generator(
    train_datagen.flow(x_train_arr, y_train, batch_size=batch_size),
    steps_per_epoch=x_train.shape[0] // batch_size,
    epochs=epochs,
    validation_data=(x_test_arr, y_test),
    callbacks=[learning_rate_reduce, checkpoint] 
)

In [ ]:
# filepath="weights_resnet.hdf5"
# checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
# learning_rate_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, verbose=1, mode='max', min_delta=0.0, cooldown=0, min_lr=0)
# early_stop = EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=25, verbose=1, mode='max', baseline=None, restore_best_weights=True)

# callbacks_list = [learning_rate_reduce, checkpoint, early_stop]
# # history = model.fit(x_train_arr, y_train, epochs= 30, batch_size = 128, validation_data = (x_test_arr, y_test), callbacks = callbacks_list) 
# history = model.fit(np.concatenate((x_train_arr, x_test_arr)), np.concatenate((y_train, y_test)), epochs= 15, batch_size = 128, validation_data = (x_test_arr, y_test), callbacks = callbacks_list) 

In [ ]:
train_pred = model.predict(x_train_arr, verbose= 1)
valid_pred = model.predict(x_test_arr, verbose= 1)

train_acc = roc_auc_score(np.round(train_pred), y_train)
valid_acc = roc_auc_score(np.round(valid_pred), y_test)

In [ ]:
confusion_matrix(np.round(valid_pred), y_test)

In [ ]:
print('Train score: '+ str(train_acc))
print('Valid score: '+ str(valid_acc))

In [ ]:
sample = pd.read_csv('../input/sample_submission.csv')
sample.head()

In [ ]:
test = []
for images in tqdm.tqdm(sample['id']):
    img = plt.imread('../input/test/test/' + images)
    test.append(img)

test = np.array(test)
print(test.shape)

In [ ]:
test = test/255

In [ ]:
test_pred = model.predict(test, verbose= 1)

In [ ]:
sample['has_cactus'] = test_pred
sample.head()

In [ ]:
sample.to_csv('sub.csv', index= False) ## 11.29 - 